In [ ]:
from pathlib import Path
import copy
import json
import networkx as nx
import osmnx as ox
# python -m pip install networkx osmnx matplotlib folium mapclassify

In [ ]:
ox.settings.use_cache = True
ox.settings.log_console = True

## 1. Fetch OSM graph of any location

In [ ]:
# get a graph
location = ('stuttgart', 'Stuttgart, Baden-Württemberg, Germany')
G = ox.graph_from_place(location[1], network_type='drive')

# impute missing edge speed and add travel times
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

In [ ]:
list(G.nodes(data=True))

In [ ]:
list(G.edges(data=True))

In [ ]:
def save(G, fname):
    nodeArr = []
    show = 10
    for node, node_data in list(G.nodes(data=True)):
        if show > 0:
            print('node:', node, node_data)
            show -= 1
        nodeArr.append((node, node_data))

    edgeArr = []
    show = 10
    for edge_a, edge_b, edge_data in list(G.edges(data=True)):
        if show > 0:
            print('edge:', edge_a, edge_b, edge_data, list(edge_data.keys()))
            show -= 1
        copied_edge_data = copy.deepcopy(edge_data)
        copied_edge_data.pop('geometry', None)
        edgeArr.append((edge_a, edge_b, copied_edge_data))

    graphDict = { 'nodes': nodeArr,
                  'edges': edgeArr}
    json.dump(graphDict, open(fname, 'w'))

save(G, Path("cache") / f'output_graph_{location[0]}.json')

## 2. Calculate shortest path between 2 OSM nodes using different weights

In [ ]:
list(G.nodes(data=True))[0]

In [ ]:
list(G.nodes(data=True))[-1]

In [ ]:
# calculate shortest path minimizing travel time
orig_node, target_node = list(G)[0], list(G)[-1]

In [ ]:
shortest_route_travle_time = nx.shortest_path(G, source=orig_node, target=target_node, weight='travel_time')
shortest_route_length = nx.shortest_path(G, source=orig_node, target=target_node, weight='length')

In [ ]:
shortest_route_travle_time

In [ ]:
route_map = ox.plot_route_folium(G, shortest_route_travle_time)
route_map

In [ ]:
route_map = ox.plot_route_folium(G, shortest_route_length)
route_map

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [ ]:
gdf_nodes

In [ ]:
gdf_edges

In [ ]:
gdf_edges.explore()

In [ ]:
G.nodes[3228724503]